In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import cv2
import sys
import math
import time
import random
import os
from datetime import datetime
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from sewar.full_ref import vifp

In [4]:
from iEBMO import bmoIE

In [5]:
pSize = [50]
maxIter = 20
# imageNameList = ['./data/kodim/kodim04.png']
# imageNameList = ['a0027.tiff', 'a0037.tiff', 'a0050.tiff']
# imageNameList = ['a0027.png', 'a0037.png', 'a0050.png']

# print(imageNameList)

imageNameList = ['./data/kodim/kodim01.png', './data/kodim/kodim02.png', './data/kodim/kodim03.png', './data/kodim/kodim04.png',
                 './data/kodim/kodim05.png', './data/kodim/kodim06.png', './data/kodim/kodim07.png', './data/kodim/kodim08.png', 
                 './data/kodim/kodim09.png', './data/kodim/kodim10.png','./data/kodim/kodim11.png', './data/kodim/kodim12.png', 
                 './data/kodim/kodim13.png', './data/kodim/kodim14.png', './data/kodim/kodim15.png', './data/kodim/kodim16.png', 
                 './data/kodim/kodim17.png', './data/kodim/kodim18.png', './data/kodim/kodim19.png', './data/kodim/kodim20.png', 
                 './data/kodim/kodim21.png', './data/kodim/kodim22.png', './data/kodim/kodim23.png', './data/kodim/kodim24.png', ]
imageNameList

['./data/kodim/kodim01.png',
 './data/kodim/kodim02.png',
 './data/kodim/kodim03.png',
 './data/kodim/kodim04.png',
 './data/kodim/kodim05.png',
 './data/kodim/kodim06.png',
 './data/kodim/kodim07.png',
 './data/kodim/kodim08.png',
 './data/kodim/kodim09.png',
 './data/kodim/kodim10.png',
 './data/kodim/kodim11.png',
 './data/kodim/kodim12.png',
 './data/kodim/kodim13.png',
 './data/kodim/kodim14.png',
 './data/kodim/kodim15.png',
 './data/kodim/kodim16.png',
 './data/kodim/kodim17.png',
 './data/kodim/kodim18.png',
 './data/kodim/kodim19.png',
 './data/kodim/kodim20.png',
 './data/kodim/kodim21.png',
 './data/kodim/kodim22.png',
 './data/kodim/kodim23.png',
 './data/kodim/kodim24.png']

In [ ]:
import datetime
for i in range(len(imageNameList)):
    averagePsnr = 0
    averageSsim = 0
    averageVif = 0
    print(imageNameList[i])
    metrics = []
    for popSize in pSize:
        print(popSize)

        inputName = "GroundTruth/kodakDataset/"+imageNameList[i]
        truthName = imageNameList[i]

        trImg = cv2.imread(truthName, 0)
        # cv2.imshow('input', trImg)
        histr = cv2.calcHist([trImg], [0], None, [256], [0, 256])
        plt.plot(histr)
        # histogram of ground truth
        name_image_list = imageNameList[i].split('/')
        time = datetime.datetime.now()
        print(time)
        # plt.savefig(f"./histogramsDIBCO/ht_{str(time).split('.')[1]}{name_image_list[-1]}") # salvar histograma da imagem de entrada
        plt.clf()
        # cv2.imwrite(f"./truthDIBCO/t_{str(time).split('.')[1]}{name_image_list[-1]}", trImg) # salvar imagem de entrada

        bestImage = deepcopy(trImg)

        maxPsnr = 0
        maxSsim = 0
        maxVif = 0
        maxfit = -1000
        for iteration in range(10):
            outputImage, fitval = bmoIE(truthName, popSize, maxIter)

            truthImage = cv2.imread(truthName, 0)
            psnrval = cv2.PSNR(outputImage, truthImage)
            ssimval = ssim(outputImage, truthImage)
            vifval = vifp(outputImage, truthImage)
            print(iteration,psnrval,ssimval,vifval,int((psnrval+ssimval+vifval)*100))
            print(psnrval, ssimval, vifval)
            
            if (psnrval+ssimval+vifval) > maxfit:
                maxfit = psnrval+ssimval+vifval
                maxPsnr = psnrval
                maxSsim = ssimval
                maxVif = vifval
                bestImage = deepcopy(outputImage)
            # cv2.imshow(str(iteration+1),bestImage)

        print(maxPsnr, maxSsim, maxVif)

        averagePsnr += maxPsnr
        averageSsim += maxSsim
        averageVif += maxVif
        print(i+1,averagePsnr,averageSsim,averageVif)
        
        ### AQUI VEM O CODIGO PARA SALVAR AS METRICAS EM ARQUIVO ###
        
        metrics = {'pSize': pSize, 'psnr': maxPsnr, 'ssim': maxSsim, 'vif': maxVif}
        metrics.append(metrics)
        
        # Gravar os dados no arquivo de texto
        with open(f'./metricas_txts/metricas_{name_image_list[-1]}.txt', 'w') as arquivo:
            for metrica in metrics:
                linha = f"pSize: {metrica['pSize']}, PSNR: {metrica['psnr']}, SSIM: {metrica['ssim']}, VIF: {metrica['vif']}\n"
                arquivo.write(linha)

        histr = cv2.calcHist([bestImage],[0],None,[256],[0,256])
        plt.plot(histr)
        time = datetime.datetime.now()
        # plt.savefig(f"./histogramsDIBCO/ho_{str(time).split('.')[1]}{name_image_list[-1]}") # salvar histograma da imagem de saida
        plt.clf()
        transImageName = "o_"+name_image_list[-1]
        # cv2.imwrite(f"./outputDIBCO/{str(time).split('.')[1]}{transImageName}", bestImage) # salvar imagem de saida


    averagePsnr /= len(imageNameList)
    averageSsim /= len(imageNameList)
    averageVif /= len(imageNameList)
    print("final")
    print(averagePsnr)
    print(averageSsim)
    print(averageVif)